<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/get_pincode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import urllib.parse

address = 'Sarjapur,bengaluru'
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response)
print(response[0]["lat"])
print(response[0]["lon"])

[{'place_id': 356967090, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 1093186415, 'boundingbox': ['12.8134543', '12.8180678', '77.7915072', '77.7948815'], 'lat': '12.8155858', 'lon': '77.79285596408383', 'display_name': "Celebrity Nature's Habitat, villa in Sarjapur, Bidaraguppe, Anekal, Bengaluru Urban District, Karnataka, India", 'class': 'landuse', 'type': 'residential', 'importance': 0.4000099999999999}, {'place_id': 3530388, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'node', 'osm_id': 456016994, 'boundingbox': ['12.8916225', '12.9316225', '77.6188622', '77.6588622'], 'lat': '12.9116225', 'lon': '77.6388622', 'display_name': 'HSR Layout, HSR Layout Ward, Bommanahalli Zone, Bengaluru, Bangalore South, Bengaluru Urban District, Karnataka, 380068, India', 'class': 'place', 'type': 'suburb', 'importance': 0.375, 'icon': 'https://nominatim.openstreetmap.org/ui/mapic

#**Postal Code from Area**

In [ ]:
import requests

def get_postal_code(area_name):
    base_url = "https://nominatim.openstreetmap.org/search"

    # Send a geocoding request to the Nominatim API
    response = requests.get(base_url, params={"q": area_name, "format": "json", "addressdetails": 1})
    data = response.json()

    if response.status_code == 200 and len(data) > 0:
        address_details = data[0].get("address")
        if address_details:
            postal_code = address_details.get("postcode")
            if postal_code:
                return postal_code

    return None

# Example usage
area_name = "bren mercury,bangalore"
postal_code = get_postal_code(area_name)
if postal_code:
    print("Postal code:", postal_code)
else:
    print("Postal code not found.")


Postal code: 560035


#**Import Libraries**

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import re
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta #
from functools import reduce

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

import gspread
from google.colab import auth
auth.authenticate_user() #

from oauth2client.client import GoogleCredentials #
#gc = gspread.authorize(GoogleCredentials.get_application_default()) #

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from gspread_dataframe import set_with_dataframe

!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def cleanText(row):
  return row.str.upper().str.strip()

def getDump(mastermerge, dump_name,start_date, end_date):
  start_date = pd.to_datetime(start_date).date()
  end_date = pd.to_datetime(end_date).date()
  file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
  reg = re.compile(".+" + dump_name+".csv")

  for (root, dirs, files) in os.walk(file_path):
      for file in files:
        path = os.path.join(root, file)
        date = str(path).split('/')[7:][0]
        date = pd.to_datetime(date).date()

        if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):
          temp = pd.read_csv(path,parse_dates=True)
          temp['DumpDate'] = date
          read_dump_name = str(path).split('/')[9:][0]
          #print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
          mastermerge = mastermerge.append(temp)

  mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
  return mastermerge

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.7 MB/s eta 0:00:00


In [ ]:
daysDelta =0

DumpDate =  pd.to_datetime((datetime.today() - timedelta(days = daysDelta)).date()).strftime('%d %B %Y')

Last30DaysDate = (datetime.today() - timedelta(days = 30 + daysDelta)).date()
Last60DaysDate = (datetime.today() - timedelta(days = 60 + daysDelta)).date()
Last90DaysDate = (datetime.today() - timedelta(days = 90 + daysDelta)).date()
Last7DaysDate = (datetime.today() - timedelta(days = 7 + daysDelta)).date()
Last45DaysDate = (datetime.today() - timedelta(days = 45 + daysDelta)).date()
MonthStartDate = datetime.today().date().replace(day=1)

start_date = Last30DaysDate.strftime('%d %B %Y')

file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"

In [ ]:
#get data from spreadsheetID and Range Name function
def get_data(spreadSheetId, rangeName):
    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

#**Postal Code from LAT LONG**

In [ ]:
import requests

def get_postal_code(latitude, longitude):
    base_url = "https://nominatim.openstreetmap.org/reverse"

    # Send a reverse geocoding request to the Nominatim API
    response = requests.get(base_url, params={"lat": latitude, "lon": longitude, "format": "json", "addressdetails": 1})
    data = response.json()

    if response.status_code == 200:
        address_details = data.get("address")
        if address_details:
            postal_code = address_details.get("postcode")
            if postal_code:
                return postal_code

    return None

# Example usage
latitude = 12.9798963  # Example latitude value
longitude = 77.7598883  # Example longitude value
postal_code = get_postal_code(latitude, longitude)



In [ ]:
key = "1oV33TLAs-QBRpAViLEq8nl2yp-t7wEiHQTl900WSG4M" #change sheet name
range = "Sheet2" #change subsheet name
wb = gc.open_by_key(key)
ws = wb.worksheet(range)
blr = get_data_df(key, range)

# blr=blr.iloc[0:50,:]

In [ ]:
# blr = blr[blr['pincode (need insights help with the rest)']=='']
blr.shape

(13, 5)

In [ ]:
# blr['Postal Code'] = blr[['Latitude','Longitude']].apply(lambda x:get_postal_code(x['Latitude'],x['Longitude']),axis=1)

In [ ]:
from joblib import Parallel, delayed

In [ ]:
blr['Postal Code'] = Parallel(n_jobs=-1,verbose=True, backend='multiprocessing')(delayed(get_postal_code)(x,y) for x,y in zip(blr['Latitude'],blr['Longitude']))

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    6.7s finished


In [ ]:
spreadSheetId = "1v57o7fms_64TKNASfMUO6nXC9dwZcMDWpY7KD3LtYQQ"
rangeName = "Sheet11"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)

ws.clear()
set_with_dataframe(ws,blr)

In [ ]:
blr

,Society,City,Address,Latitude,Longitude,Postal Code
0,33 LE,Vadodara,"33 LE, Vadodara",22.2917025,73.121618,390001
1,Aarcon Abhay,Vadodara,"Aarcon Abhay, Vadodara",22.326867,73.198066,390001
2,Aatmiya Heritage,Vadodara,"Aatmiya Heritage, Vadodara",22.2269157,73.1823948,390001
3,Aditya Elanza,Vadodara,"Aditya Elanza, Vadodara",22.2712069,73.1344812,390001
4,Aditya Villa,Vadodara,"Aditya Villa, Vadodara",22.2853812,73.2445712,390001
5,Anand Garden,Vadodara,"Anand Garden, Vadodara",22.3233181,73.1287922,390001
6,Antica Green Woods,Vadodara,"Antica Green Woods, Vadodara",22.3293548,73.1142046,390001
7,ASOPALAV ONE WEST MEMBERS ASSOCIATION,Vadodara,"ASOPALAV ONE WEST MEMBERS ASSOCIATION, Vadodara",22.3071588,73.1812187,390005
8,Atlantis Villas,Vadodara,"Atlantis Villas, Vadodara",22.3557795,73.1726558,390001
9,Auro Heights,Vadodara,"Auro Heights, Vadodara",22.2594682,73.1724348,390001
